In [1]:
from officelib.wordlib import Word, wdc
from officelib.xllib import Excel, xlc

In [2]:
w = Word()

In [3]:
d = w.ActiveDocument

In [19]:
def iter_tables(d, first):
    for i, table in enumerate(d.Tables, 1):
        if i >= first:
            yield table
def iter_rows(t, first):
    for i, row in enumerate(t.Rows, 1):
        if i >= first:
            yield row

In [5]:
def text(r):
    raw = r.Text.rstrip(" \r\x07")
    
    # \r -> \n
    return raw.replace("\r", "\n")

def indented(s, n):
    ind = " " * n
    parts = s.splitlines()
    return "\n".join(ind + p for p in parts)

# for t in iter_tables(d, 1):
#     print(repr(text(t.Rows(2).Cells(1).Range)))
#     for r in iter_rows(t, 3):
#         r = text(r.Cells(2).Range)
#         print(indented(r, 4))

In [6]:
xl = Excel()
wb = xl.ActiveWorkbook or xl.Workbooks.Add()
ws = wb.ActiveSheet or wb.Worksheets.Add()
cells = ws.Cells
cr = cells.Range

In [54]:
# used for initial import

# start = cr("B3")
# n = 0
# for i, t in enumerate(d.Tables, 1):
#     for i2, r in enumerate(t.Rows, 1):
#         if i2 == 1: continue
#         n += 1
#         for i3, cell in enumerate(r.Cells, 1):
#             if i3 == 1: continue
#             xcell = start.GetOffset(n - 1, i3 - 1)
#             xcell.NumberFormat = "@"
#             xcell.Value2 = text(cell.Range)
            
# for c in (4, 5, 6, 7):
#     ws.Columns(c).ColumnWidth = 40

In [23]:
def pmg(o):
    return sorted(o._prop_map_get_.keys())

In [8]:
# 6/11/2020 update FIH columns

In [31]:
# action = cr("I2")
# assign = cr("J2")
# due = cr("K2")

# w_action = 8
# w_due = 9

# def _itemize(line):
#     if line and not line.isspace():
#         return "- " + line
#     return line

# def itemize(text):
#     parts = text.splitlines()
#     parts2 = [_itemize(l) for l in parts]
#     return "\n".join(parts2)

# off = 0
# for t in d.Tables:
#     for i, row in enumerate(t.Rows, 1):
#         if i == 1: continue # skip header
            
#         off += 1
        
#         r_action = row.Cells(w_action).Range
# #         r_assign = 
# #         m = r.TextRetrievalMode
# #         m.IncludeFieldCodes = False
# #         m.IncludeHiddenText = False
# #         m.ViewType = wdc.wdPrintView
# #         del m,r
        
#         t_action = text(r_action).strip()
#         t_assign = text(row.Cells(w_due).Range).strip()
#         xaction = action.GetOffset(off, 0)
#         xassign = assign.GetOffset(off, 0)
        
#         xaction.Value2 = itemize(t_action)
#         xassign.Value2 = itemize(t_assign)
        
# #         print(t_action, t_assign)
# #         print(xaction.GetAddress(0,0), xassign.GetAddress(0,0))
# #         print("-----------------")

Action: plan to be generated 
Create inventory list of IT systems
Categorize

Feed into #2 Nathan – pt person

Joel – support (generate list)

Draft 6/30
Final July
I3 J3
-----------------
Perform after #1. Action: assess IT list Get quote from USDM to create procedure for security Nathan, Sabrina,

 Support: Joel

Assessment of scope after list in hand, by
I4 J4
-----------------
Determine if own in house or USDM.  Get quote from USDM NS
I5 J5
-----------------
USDM Quote? YES  

Full GMP -Yes 
I6 J6
-----------------
No 
I7 J7
-----------------
Full GMP

USDM- no 
I8 J8
-----------------
Full GMP – determine how much is covered by other security controls

USDM- no 
I9 J9
-----------------
Low risk vs other obs. Need to ID GMP systems in scope and determine reqts for Full GMP.  Incl GSS and SOP for GSS.

USDM- no 
I10 J10
-----------------
Need clarification from USDM -check final report to see if clearer 
I11 J11
-----------------
Sabrina to outline plan.

Notes: Create data policy a

In [34]:
# create actions table
class Worksheet:
    def __init__(self, ws):
        self.ws = ws
        self.cells = ws.Cells
        self.cr = self.cells.Range

In [57]:
findings = Worksheet(wb.Worksheets("Findings"))
actions = Worksheet(wb.Worksheets("Actions"))

fact = findings.cr("I2")
fid = findings.cr("B2")
fdue = findings.cr("J2")
fwho = findings.cr("K2")
fgact = findings.cr("M2")
frisk = findings.cr("G2")
fprio = findings.cr("H2")

rmap = {
    "High": 3,
    "Medium": 2,
    "Low": 1
}

def rank(risk, prio):
    r = rmap[risk]
    return (r - 1) * 3 + 4 - prio

aact = actions.cr("C2")
aid = actions.cr("B2")
adue = actions.cr("F2")
awho = actions.cr("G2")
atgt = actions.cr("E2")
ascore = actions.cr("D2")

end = 29 - 2
print(fact.GetOffset(end, 0).GetAddress(0,0))

I29


In [58]:
def _tget(i, h):
    if h is None:
        return ""
    t = h.GetOffset(i, 0).Value2
    return t or ""

def _put(i, c, v):
    c.GetOffset(i, 0).Value2 = v
    
def _irrank(i, risk, prio):
    r = _tget(i, risk)
    p = _tget(i, prio)
    return rank(r, p)

def _tget_who(i, who):
    t = _tget(i, who)
    t = t or ""
    t = t.replace("- ", "").replace("\n\n", "\n")
    return t

def do_act(target, offs, i, id, tact, who, due, risk, prio):
    offs += 1
    
    twho = _tget_who(i, who)
    tdue = _tget(i, due)
    tid = _tget(i, id)
    trank = _irrank(i, risk, prio)
    
    _put(offs, aact, tact)
    _put(offs, aid, tid)
    _put(offs, adue, tdue)
    _put(offs, awho, twho)
    _put(offs, atgt, target)
    _put(offs, ascore, trank)
    return offs
    
def _getacts(i, act):
    t = _tget(i, act)
    if t in ("", "N/A"):
        return []
    
    acts = []
    for l in t.splitlines():
        x = l.strip()
        if x:
            acts.append(x.lstrip("- "))
    return acts
    
offs = 0
for i in range(1, end):
    
    acts = _getacts(i, fact) 
    for a in acts:
        offs = do_act("FIH", offs, i, fid, a, fwho, fdue, frisk, fprio)
    
    gacts = _getacts(i, fgact)
    for a in gacts:
        offs = do_act("GMP", offs, i, fid, a, None, None, frisk, fprio)
    

In [50]:
for c in xl.Selection:
    v = c.Value2
    if v is not None:
        v = v.replace("\n\n", "\n")
        v = v.replace("- ","")
        c.Value2 = v